In [11]:
import os
import pandas as pd
import torch
import numpy as np
import pydicom
from torch.utils.data import DataLoader, Dataset
from monai.networks.nets import resnet
import torch.nn.functional as F
from collections import defaultdict

# ——— 1. DICOM 序列读取 & 归一化 —————————————————————
def load_dicom_series(folder):
    # 1) 递归搜所有 .dcm 文件
    dcm_files = []
    for root, _, files in os.walk(folder):
        for fn in files:
            if fn.lower().endswith('.dcm'):
                dcm_files.append(os.path.join(root, fn))
    if not dcm_files:
        raise ValueError(f"No DICOMs found in {folder}")

    # 2) 读每个文件的 pixel_array.shape，按它分组
    shape_groups = defaultdict(list)
    for fp in dcm_files:
        ds  = pydicom.dcmread(fp)
        img = ds.pixel_array  # 实际 ndarray
        shape_groups[img.shape].append(fp)

    # 3) 选出文件数最多的那一组
    best_shape, best_files = max(
        shape_groups.items(), key=lambda x: len(x[1])
    )
    print(f"→ 选用实际像素尺寸 {best_shape}，共 {len(best_files)} 张切片")

    # 4) 读取并按 InstanceNumber 排序
    slices = []
    for fp in best_files:
        ds   = pydicom.dcmread(fp)
        inst = getattr(ds, "InstanceNumber", 0)
        img  = ds.pixel_array.astype(np.float32)
        slices.append((inst, img))
    slices.sort(key=lambda x: x[0])

    # 5) 堆叠成 3D 体（D, H, W）
    vol = np.stack([s[1] for s in slices], axis=0)

    # 6) 归一化到 [0,1]
    vol = (vol - vol.min()) / (vol.max() - vol.min())
    return vol

# ——— 2. 3D-ResNet 特征提取函数 ————————————————————
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net3d = resnet.resnet10(spatial_dims=3, n_input_channels=1, num_classes=1)
net3d.fc = torch.nn.Identity()
net3d = net3d.to(device).eval()

def extract_features_3d(vol_np):
    # vol_np: ndarray (D,H,W) in [0,1]
    t = torch.from_numpy(vol_np).unsqueeze(0).unsqueeze(0).to(device)  
    # now shape (1,1,D,H,W)
    # resize to, say, (1,1,64,128,128):
    t = F.interpolate(t, size=(64,128,128),
                      mode='trilinear', align_corners=False)
    with torch.no_grad():
        feat = net3d(t)       # (1,512)
    return feat.cpu().numpy().squeeze()  # (512,)

# ——— 3. 批量处理所有患者 ————————————————————————
root_dir = "/mnt/e/Block_2/Machine_Learning/ASS_Part2/Project/testdata/dataset2"
case_ids = [d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d))]

features = []
ids       = []
for cid in case_ids:
    # 找到 CT 文件夹（假设名字里含 "CT"）
    case_dir = os.path.join(root_dir, cid)
    series = next((d for d in os.listdir(case_dir) if "CT" in d), None)
    if series is None:
        print(f"{cid} 没找到 CT 文件夹，跳过")
        continue

    vol = load_dicom_series(os.path.join(case_dir, series))
    feat = extract_features_3d(vol)  
    features.append(feat)
    ids.append(cid)
    print(f"{cid} 提取完毕")

# ——— 4. 保存成 DataFrame 便于后续合并 —————————————
df_img = pd.DataFrame(
    features,
    index=ids,
    columns=[f"img_feat_{i:03d}" for i in range(features[0].shape[0])]
)
df_img.to_csv("../result/image_features_3d.csv")
print("所有影像特征已保存到 ../result/image_features_3d.csv")


→ 选用实际像素尺寸 (192, 192)，共 526 张切片
✔️ AMC-001 提取完毕
→ 选用实际像素尺寸 (512, 512)，共 271 张切片
✔️ AMC-002 提取完毕
→ 选用实际像素尺寸 (512, 512)，共 567 张切片
✔️ R01-006 提取完毕
→ 选用实际像素尺寸 (512, 512)，共 483 张切片
✔️ R01-023 提取完毕
→ 选用实际像素尺寸 (512, 512)，共 531 张切片
✔️ R01-024 提取完毕
→ 选用实际像素尺寸 (512, 512)，共 376 张切片
✔️ R01-026 提取完毕
→ 选用实际像素尺寸 (512, 512)，共 1087 张切片
✔️ R01-028 提取完毕
所有影像特征已保存到 ../result/image_features_3d.csv


In [15]:
import pandas as pd
import numpy as np

from sklearn.ensemble        import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.compose         import ColumnTransformer
from sklearn.pipeline        import Pipeline
from sklearn.impute          import SimpleImputer
from sklearn.preprocessing   import OneHotEncoder

# 1. 读取三类特征矩阵
df_img  = pd.read_csv("../result/image_features_3d.csv", index_col=0)           # 样本×512
df_rna  = pd.read_csv("../result/rnaseq_processed.csv", index_col=0)            # 样本×n_genes
df_clin = pd.read_csv("../result/clinical2_processed.csv", index_col="Case ID") # 样本×临床（含 Survival Status）

# 2. 取交集样本
common = df_clin.index.intersection(df_rna.index).intersection(df_img.index)
print(f"共同样本数：{len(common)}")

img_sub  = df_img.loc[common]
rna_sub  = df_rna.loc[common]
clin_sub = df_clin.loc[common]

# 3. 构造标签 y
y = (clin_sub["Survival Status"] == "Dead").astype(int).values

# 4. 处理临床特征：填补 + One-Hot
#    4.1 分离数值列和分类列（去掉 Survival Status）
clin_feats = clin_sub.drop(columns="Survival Status")
num_cols = clin_feats.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_cols = clin_feats.select_dtypes(include=["object"]).columns.tolist()

#    4.2 构建预处理流水线
pre_clin = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_cols),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ]), cat_cols)
], remainder="drop")

#    4.3 得到纯数值的临床矩阵
X_clin_ohe = pre_clin.fit_transform(clin_feats)

# 5. 构造三种特征集
X_img     = img_sub.values            # (N, 512)
X_rna     = rna_sub.values            # (N, n_genes)
X_rnaclin = np.hstack([X_rna, X_clin_ohe])         # (N, n_genes + n_clin_ohe)
X_all     = np.hstack([X_img, X_rna, X_clin_ohe])  # (N, 512 + n_genes + n_clin_ohe)

# 6. 模型与 CV 配置
cv      = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
clf     = RandomForestClassifier(
              n_estimators=100,
              min_samples_leaf=3,
              random_state=42,
              n_jobs=-1
          )
scoring = ["f1", "roc_auc", "precision", "recall"]

# 7. 评估函数
def eval_model(X, name):
    scores = cross_validate(clf, X, y,
                            cv=cv,
                            scoring=scoring,
                            return_train_score=False)
    print(f"\n=== {name} ===")
    for m in scoring:
        arr = scores[f"test_{m}"]
        print(f"{m:9s}: {arr.mean():.3f} ± {arr.std():.3f}")

# 8. 分别评估三种方案
eval_model(X_img,     "Image Only")
eval_model(X_rnaclin, "RNA + Clinical")
eval_model(X_all,     "Image + RNA + Clinical")

共同样本数：5


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



=== Image Only ===
f1       : 0.250 ± 0.250
roc_auc  : 0.500 ± 0.000
precision: 0.167 ± 0.167
recall   : 0.500 ± 0.500


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



=== RNA + Clinical ===
f1       : 0.250 ± 0.250
roc_auc  : 0.500 ± 0.000
precision: 0.167 ± 0.167
recall   : 0.500 ± 0.500

=== Image + RNA + Clinical ===
f1       : 0.250 ± 0.250
roc_auc  : 0.500 ± 0.000
precision: 0.167 ± 0.167
recall   : 0.500 ± 0.500


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


In [20]:
import os, pydicom, numpy as np, SimpleITK as sitk

# get 3D volume
def read_dicom_series(folder_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(folder_path)
    reader.SetFileNames(dicom_names)
    return reader.Execute()

# get CT Series UID from segmentation
def get_series_uid_from_seg(seg_path):
    return pydicom.dcmread(seg_path).ReferencedSeriesSequence[0].SeriesInstanceUID

# matching UID
def find_ct_folder_by_uid(folder, uid):
    for root, _, files in os.walk(folder):
        for f in files:
            if f.lower().endswith(".dcm"):
                try:
                    if pydicom.dcmread(os.path.join(root, f), stop_before_pixels=True).SeriesInstanceUID == uid:
                        return root
                except: continue
    return None

# calculate Z-coordinate
def get_z_coords(sitk_img):
    sz, sp, org, dir3 = sitk_img.GetSize(), sitk_img.GetSpacing(), sitk_img.GetOrigin(), sitk_img.GetDirection()
    return [org[2] + i * sp[2] * dir3[8] for i in range(sz[2])]

# matching z
def align_segmentation_by_z(seg_img, seg_z, ct_z):
    zset = set(round(z, 1) for z in ct_z)
    return sitk.GetArrayFromImage(seg_img)[[i for i, z in enumerate(seg_z) if round(z, 1) in zset]]

# resample（1mm*3）
def resample_to_spacing(image_sitk, new_spacing=(1.0, 1.0, 1.0), interpolator=sitk.sitkLinear):
    original_spacing = image_sitk.GetSpacing()
    original_size = image_sitk.GetSize()
    new_size = [
        int(round(original_size[i] * (original_spacing[i] / new_spacing[i])))
        for i in range(3)
    ]
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputDirection(image_sitk.GetDirection())
    resampler.SetOutputOrigin(image_sitk.GetOrigin())
    resampler.SetInterpolator(interpolator)
    return resampler.Execute(image_sitk)

# normalise to [0,1]
def normalize_gray_roi_adaptive(image_np, mask_np, min_percentile=1, max_percentile=99):
    roi = image_np[mask_np > 0]
    if roi.size == 0:
        return np.zeros_like(image_np)
    vmin, vmax = np.percentile(roi, [min_percentile, max_percentile])
    image_clipped = np.clip(image_np, vmin, vmax)
    return (image_clipped - vmin) / (vmax - vmin + 1e-6)

# main
def load_aligned_ct_and_mask(patient_folder, target_spacing=(1.5, 1.5, 1.5)):
    seg_path = next((os.path.join(root, f) for root, _, files in os.walk(patient_folder)
                     for f in files if "segmentation" in root.lower() and f.endswith(".dcm")), None)
    if not seg_path: return None, None, None

    uid = get_series_uid_from_seg(seg_path)
    ct_folder = find_ct_folder_by_uid(patient_folder, uid)
    if not ct_folder: return None, None, None

    ct_img = read_dicom_series(ct_folder)
    seg_img = sitk.ReadImage(seg_path)

    # Resample to same spacing (1mm)
    ct_img = resample_to_spacing(ct_img, new_spacing=target_spacing)
    
    # resample segmentation
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ct_img)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    seg_img = resampler.Execute(seg_img)

    # z coordinate alignment
    ct_z = get_z_coords(ct_img)
    seg_z = get_z_coords(seg_img)
    mask = align_segmentation_by_z(seg_img, seg_z, ct_z)

    image = sitk.GetArrayFromImage(ct_img)
    image = normalize_gray_roi_adaptive(image, mask)

    # output
    return image, mask, target_spacing

base_dir = "../testdata/dataset2"

for patient_id in sorted(os.listdir(base_dir)):
    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue  # skip non-folders

    image, mask, spacing = load_aligned_ct_and_mask(patient_path)

    if image is not None:
        roi = image * (mask > 0)
        print(f"{patient_id} ROI volume shape: {roi.shape}")
    else:
        print(f"{patient_id} fail")

# Extraction of tumour 3D shape features based on marching cubes algorithm
import os
import numpy as np
import pandas as pd
import skimage as ski
from skimage import io, data, filters, measure, morphology
import matplotlib
import matplotlib.pyplot as plt
import nibabel as nib
import skimage.measure
from scipy.spatial.distance import  pdist

def extract_shape_features(mask, spacing=(1.0, 1.0, 1.0)):
    verts, faces, _, _ = measure.marching_cubes(mask, level=0.5, spacing=spacing)

    # surface area
    surface_area = measure.mesh_surface_area(verts, faces)

    # volumetric
    voxel_volume = np.prod(spacing)
    volume = np.sum(mask > 0) * voxel_volume

    # maximum diameter
    if len(verts) >= 2:
        max_diameter = pdist(verts).max()
    else:
        max_diameter = 0.0

    # Compactness
    if volume > 0:
        compactness = (surface_area ** 3) / (volume ** 2)
    else:
        compactness = 0.0

    return {
        "volume_mm3": volume,
        "surface_mm2": surface_area,
        "max_diameter_mm": max_diameter,
        "compactness": compactness
    }

for patient_id in sorted(os.listdir(base_dir)):
    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue

    image, mask, spacing = load_aligned_ct_and_mask(patient_path)
    if image is None or mask is None:
        print(f"{patient_id} skip")
        continue

    features = extract_shape_features(mask, spacing)
    print(f"{patient_id} features:", features)

# GLCM
def gray_level_cooccurrence_features(img, mask, levels=32):
    bin_img = (img * (levels - 1)).astype(np.uint8)
    glcm = _calculate_glcm2(bin_img, mask, levels)
    glcm = glcm / np.sum(glcm)

    ix = np.arange(1, levels+1)[:, None, None].astype(np.float64)
    iy = np.arange(1, levels+1)[None, :, None].astype(np.float64)

    ux = np.mean(glcm, axis=0, keepdims=True)
    uy = np.mean(glcm, axis=1, keepdims=True)
    sigma_x = np.std(glcm, axis=0, keepdims=True)
    sigma_y = np.std(glcm, axis=1, keepdims=True)

    # downscale protection
    sigma_x[sigma_x < 1e-3] = 1e-3
    sigma_y[sigma_y < 1e-3] = 1e-3

    features = {
        "contrast": np.mean(np.sum((ix - iy) ** 2 * glcm, axis=(0, 1))),
        "correlation": np.mean(np.sum((ix * iy * glcm - ux * uy) / (sigma_x * sigma_y + 1e-6), axis=(0, 1))),
        "dissimilarity": np.mean(np.sum(np.abs(ix - iy) * glcm, axis=(0, 1))),
        "homogeneity": np.mean(np.sum(glcm / (1 + np.abs(ix - iy)), axis=(0, 1))),
    }

    return features

# GLCM2
def _calculate_glcm2(img, mask, nbins):
    out = np.zeros((nbins, nbins, 13))
    offsets = [
        (1, 0, 0), (0, 1, 0), (0, 0, 1),
        (1, 1, 0), (-1, 1, 0), (1, 0, 1),
        (-1, 0, 1), (0, 1, 1), (0, -1, 1),
        (1, 1, 1), (-1, 1, 1), (1, -1, 1), (1, 1, -1)
    ]
    matrix = np.array(img)
    matrix[mask <= 0] = nbins
    s = matrix.shape
    bins = np.arange(0, nbins + 1)

    for i, offset in enumerate(offsets):
        matrix1 = np.ravel(matrix[max(offset[0], 0):s[0]+min(offset[0], 0),
                                  max(offset[1], 0):s[1]+min(offset[1], 0),
                                  max(offset[2], 0):s[2]+min(offset[2], 0)])

        matrix2 = np.ravel(matrix[max(-offset[0], 0):s[0]+min(-offset[0], 0),
                                  max(-offset[1], 0):s[1]+min(-offset[1], 0),
                                  max(-offset[2], 0):s[2]+min(-offset[2], 0)])

        try:
            out[:, :, i] = np.histogram2d(matrix1, matrix2, bins=bins)[0]
        except Exception as e:
            print(f"GLCM histogram failed for offset {offset}: {e}")
            continue

    return out

import os
import pandas as pd

base_dir = "../testdata/dataset2"
all_features = []

for patient_id in sorted(os.listdir(base_dir)):
    if not patient_id.startswith("R01-"):
        continue

    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue

    image, mask, spacing = load_aligned_ct_and_mask(patient_path)

    if image is not None and mask is not None:
        image = normalize_gray_roi_adaptive(image, mask)
        glcm = gray_level_cooccurrence_features(image, mask)

        glcm["patient_id"] = patient_id
        all_features.append(glcm)
    else:
        print(f"{patient_id} skip")

import os
import pandas as pd
import numpy as np

base_dir = "../testdata/dataset2"
all_features = []

for patient_id in sorted(os.listdir(base_dir)):
    if not patient_id.startswith("R01-"):
        continue

    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue

    image, mask, spacing = load_aligned_ct_and_mask(patient_path)
    if image is None or mask is None:
        print(f"{patient_id} skip")
        continue

    image = normalize_gray_roi_adaptive(image, mask)
    shape_feats = extract_shape_features(mask, spacing)
    glcm_feats = gray_level_cooccurrence_features(image, mask)

    features = {"patient_id": patient_id}
    features.update(shape_feats)
    features.update(glcm_feats)
    all_features.append(features)

# Save the result as a csv file
output_path = "../result/combined_features.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df = pd.DataFrame(all_features)
df.to_csv("../result/features2.csv", index=False)
print(f"Done,{len(df)} in total")

AMC-001 fail
AMC-002 fail
R01-006 ROI volume shape: (236, 333, 333)
R01-023 ROI volume shape: (201, 287, 287)
R01-024 ROI volume shape: (221, 280, 280)
R01-026 ROI volume shape: (251, 273, 273)
R01-028 ROI volume shape: (220, 293, 293)
AMC-001 skip
AMC-002 skip
R01-006 features: {'volume_mm3': 4039.875, 'surface_mm2': 2238.3950887495143, 'max_diameter_mm': 31.4955010692827, 'compactness': 687.1861180282815}
R01-023 features: {'volume_mm3': 59032.125, 'surface_mm2': 16694.84371499259, 'max_diameter_mm': 87.84357783922867, 'compactness': 1335.2735296617432}
R01-024 features: {'volume_mm3': 4826.25, 'surface_mm2': 2425.9580374564475, 'max_diameter_mm': 44.84540438975412, 'compactness': 612.95730355034}
R01-026 features: {'volume_mm3': 3847.5, 'surface_mm2': 1861.9637920419855, 'max_diameter_mm': 29.00281781556549, 'compactness': 436.0702282359349}
R01-028 features: {'volume_mm3': 29764.125, 'surface_mm2': 7040.110005083245, 'max_diameter_mm': 60.01313895818942, 'compactness': 393.86926827

In [22]:
import pandas as pd
import numpy as np

from sklearn.compose         import ColumnTransformer
from sklearn.pipeline        import Pipeline
from sklearn.impute          import SimpleImputer
from sklearn.preprocessing   import OneHotEncoder, StandardScaler
from sklearn.ensemble        import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate

# 1. 读取特征矩阵
df_img  = pd.read_csv("../result/features2.csv",       index_col="patient_id")  # 手动提取的影像特征
df_rna  = pd.read_csv("../result/rnaseq_processed.csv", index_col=0)            # 预处理好的 RNA
df_clin = pd.read_csv("../result/clinical2_processed.csv",
                      index_col="Case ID")                                      # 预处理好的临床（含 Survival Status）

# 2. 取交集样本
common = df_img.index.intersection(df_rna.index).intersection(df_clin.index)
print(f"共同样本数：{len(common)}")

img_sub  = df_img.loc[common]
rna_sub  = df_rna.loc[common]
clin_sub = df_clin.loc[common]

# 3. 构造标签 y（Dead=1, Alive=0）
y = (clin_sub["Survival Status"] == "Dead").astype(int).values

# 4. 临床预处理：中位数/众数填补 + One-Hot
#    分离数值列和分类列（去掉 Survival Status）
clin_feats = clin_sub.drop(columns="Survival Status")
num_cols = clin_feats.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_cols = clin_feats.select_dtypes(include=["object"]).columns.tolist()

pre_clin = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_cols),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ]), cat_cols)
], remainder="drop")

X_clin_ohe = pre_clin.fit_transform(clin_feats)  # shape = (N, n_clin_ohe)

# 5. RNA 和影像标准化
X_rna_scaled = StandardScaler().fit_transform(rna_sub.values)
X_img_scaled = StandardScaler().fit_transform(img_sub.values)

# 6. 构造四种特征集
X_img_only     = X_img_scaled
X_rna_only     = X_rna_scaled
X_clin_only    = X_clin_ohe
X_all_three    = np.hstack([X_img_scaled, X_rna_scaled, X_clin_ohe])

# 7. 模型 & CV & 指标
cv      = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
clf     = RandomForestClassifier(n_estimators=100,
                                 min_samples_leaf=3,
                                 random_state=42,
                                 n_jobs=-1)
scoring = ["f1", "roc_auc", "precision", "recall"]

# 用来收集各模型结果
results = []

def eval_model(X, name):
    scores = cross_validate(clf, X, y,
                            cv=cv,
                            scoring=scoring,
                            return_train_score=False)
    # 组装当前模型的各项指标均值和标准差
    row = {"model": name}
    for m in scoring:
        arr = scores[f"test_{m}"]
        row[f"{m}_mean"] = arr.mean()
        row[f"{m}_std"]  = arr.std()
    results.append(row)
    # 同时打印到控制台
    print(f"\n=== {name} ===")
    for m in scoring:
        print(f"{m:9s}: {row[f'{m}_mean']:.3f} ± {row[f'{m}_std']:.3f}")

# 8. 依次评估
eval_model(X_img_only,  "Image Only")
eval_model(X_rna_only,  "RNA Only")
eval_model(X_clin_only, "Clinical Only")
eval_model(X_all_three, "Image + RNA + Clinical")

# 9. 保存所有结果到 CSV
df_res = pd.DataFrame(results)
os.makedirs("result", exist_ok=True)
df_res.to_csv("result/result2.csv", index=False)
print(f"save as result/result2.csv")


共同样本数：5


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



=== Image Only ===
f1       : 0.250 ± 0.250
roc_auc  : 0.500 ± 0.000
precision: 0.167 ± 0.167
recall   : 0.500 ± 0.500


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



=== RNA Only ===
f1       : 0.250 ± 0.250
roc_auc  : 0.500 ± 0.000
precision: 0.167 ± 0.167
recall   : 0.500 ± 0.500


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



=== Clinical Only ===
f1       : 0.250 ± 0.250
roc_auc  : 0.500 ± 0.000
precision: 0.167 ± 0.167
recall   : 0.500 ± 0.500

=== Image + RNA + Clinical ===
f1       : 0.250 ± 0.250
roc_auc  : 0.500 ± 0.000
precision: 0.167 ± 0.167
recall   : 0.500 ± 0.500


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
